In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

lc=LabelEncoder()

In [3]:
df= pd.read_csv('Student_Performance.csv')
df[:5]

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0


In [4]:
y=df.iloc[:,-1]
x=df.iloc[:,0:5]
label = lc.fit_transform(df['Extracurricular Activities'])
x.drop('Extracurricular Activities',axis=1,inplace=True)
x['Extracurricular Activities']=list(label)
x[:5]

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Extracurricular Activities
0,7,99,9,1,1
1,4,82,4,2,0
2,8,51,7,2,1
3,5,52,5,2,1
4,7,75,8,5,0


In [7]:
Xtrain,Xtest,ytrain,ytest = train_test_split(np.array(x),np.array(y),test_size=0.2,random_state=42)
Xtest,Xval,ytest,yval = train_test_split(Xtest,ytest,test_size=0.5,random_state=42)

In [8]:
class Customdataset(Dataset):
    def __init__(self,a,b):
        self.x=torch.tensor(a,dtype=torch.float32)
        self.y=torch.tensor(b,dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index],self.y[index]
    
traindataset=Customdataset(a=Xtrain,b=ytrain)
valdataset=Customdataset(a=Xval,b=yval)
testdataset=Customdataset(a=Xtest,b=ytest)

traindl=DataLoader(traindataset,batch_size=16,shuffle=True)
testdl=DataLoader(testdataset,batch_size=16,shuffle=True)
valdl=DataLoader(valdataset,batch_size=16,shuffle=True)

In [53]:
class regression(nn.Module):
    def __init__(self):
        super(regression,self).__init__()
        self.Linear1=nn.Linear(x.shape[1],20)
        self.Linear2=nn.Linear(20,1)

    def forward(self,x):
        x=self.Linear1(x)
        x=self.Linear2(x)
        return x

model=regression()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
critetion=nn.MSELoss()

In [60]:
for i in range(3):
    l1,l2=0,0
    for values,labels in traindl:
        ypred = model(values)
        print(values)
        print(ypred)
        print(labels)
        tloss = critetion(ypred,labels.unsqueeze(1))
        optimizer.zero_grad()
        tloss.backward()        
        optimizer.step()
        l1=labels.unsqueeze(1)
        break
    break
    print(mean_absolute_error(l1,ypred.detach().numpy()))     
    with torch.no_grad():
        for values,labels in valdl:
            ypred = model(values)
            tloss = critetion(ypred,labels.unsqueeze(1))
            l2=labels.unsqueeze(1)
        print(mean_absolute_error(l1,ypred))
    break
                

tensor([[ 3., 90.,  8.,  3.,  1.],
        [ 3., 66.,  6.,  7.,  1.],
        [ 8., 90.,  9.,  5.,  0.],
        [ 6., 94.,  6.,  4.,  0.],
        [ 6., 49.,  5.,  1.,  1.],
        [ 4., 43.,  8.,  5.,  0.],
        [ 7., 94.,  4.,  4.,  1.],
        [ 9., 77.,  6.,  1.,  0.],
        [ 9., 42.,  9.,  3.,  1.],
        [ 2., 98.,  5.,  3.,  0.],
        [ 1., 93.,  6.,  2.,  0.],
        [ 4., 44.,  6.,  8.,  1.],
        [ 9., 63.,  4.,  4.,  1.],
        [ 6., 71.,  4.,  1.,  0.],
        [ 2., 80.,  9.,  5.,  0.],
        [ 7., 87.,  7.,  8.,  0.]])
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
tensor([70., 48., 84., 83., 35., 27., 84., 68., 37., 71., 65., 24., 59., 59.,
        55., 76.])
